In [1]:
import os
import sys
import time
from pathlib import Path

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

In [2]:
import pandas as pd
from PIL import Image, ImageDraw
import copy

In [3]:
from services.xml_document_info import (get_xml_info, get_xml_image_info)
from services.get_xml import  create_pdf_processing_paths, extract_pdf_metadata, process_input_pdf
from anuvaad_auditor.loghandler import log_info
from anuvaad_auditor.loghandler import log_error
from src.services import main
from src.services.get_underline import get_underline
from services import get_xml
from src.services.child_text_unify_to_parent import ChildTextUnify
from services.preprocess import prepocess_pdf_regions
from services.get_tables import page_num_correction , get_text_table_line_df
from src.services.ocr_text_utilities import  tesseract_ocr
from src.services.get_response import process_image_df,  process_table_df, df_to_json, process_line_df, process_bg_image

from utilities.filesystem import (create_directory, read_directory_files)
import config

import src.utilities.app_context as app_context
app_context.init()

In [4]:
def draw_box(filepath, desired_width, desired_height, df, color="green", save=False):
    image  = Image.open(filepath)
    image  = image.resize((desired_width, desired_height))
    draw   = ImageDraw.Draw(image)
    
    for index, row in df.iterrows():
        left   = int(row['text_left'])
        right  = int(row['text_width'] + left)
        top    = int(row['text_top'])
        bottom = int(row["text_height"] + top)
        
        draw.rectangle(((left, top), (right,bottom)), outline=color)
    save_filepath = os.path.join(os.path.dirname(filepath), 'processed_' + os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

def show_df(df):
    return df.head(df.shape[0])

## start of the program

In [5]:
'''
  folder structure of test data goes like this
  - notebooks
      - sample-data
          - input
          - output
          
  the pdfs are present in "input" directory and they are *.pdf is added into .gitignore
  just to save repo size.
'''

base_dir   = os.getcwd()
input_dir  = os.path.join(base_dir, 'sample-data','input')
save_dir   = os.path.join(base_dir, 'sample-data', 'bbox_output')
output_dir = os.path.join(base_dir, 'sample-data', 'output')

filename   = '20.03.2020.pdf'
# filename  = 'Madras_HC_02.pdf'


In [6]:
from src.services import main

In [ ]:
# running for all
files = read_directory_files(input_dir, "*.pdf")
for file in files:
    app_context.application_context = dict({'task_id': 'BM-DUMMY', 'filename': os.path.basename(file)})
    data       = main.DocumentStructure(app_context, os.path.basename(file), "en", base_dir=input_dir)


[2020-09-09 13:00:34,427] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing <module 'src.utilities.app_context' from '/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/app_context.py'>
[2020-09-09 13:00:34,428] {loghandler.py:20} MainThread INFO in loghandler: document preprocessing started ===>
[2020-09-09 13:00:34,439] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/000_en_Presidents_Secretariat_2020-04-25_59275b40-f26e-11ea-b442-acde48001122
[2020-09-09 13:00:34,440] {loghandler.py:20} MainThread INFO in loghandler: filepath /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/000_en_Presidents_Secretariat_2020-04-25.pdf, working_dir /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extract

/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/services/child_text_unify_to_parent.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[block_index]['text'] = str(text)


[2020-09-09 13:00:36,341] {loghandler.py:20} MainThread INFO in loghandler: DocumentStructure : successfully received blocks in json response
[2020-09-09 13:00:36,343] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing <module 'src.utilities.app_context' from '/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/app_context.py'>
[2020-09-09 13:00:36,345] {loghandler.py:20} MainThread INFO in loghandler: document preprocessing started ===>
[2020-09-09 13:00:36,346] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/000_hi_Presidents_Secretariat_2020-04-25_5a4bcc22-f26e-11ea-a19d-acde48001122
[2020-09-09 13:00:36,348] {loghandler.py:20} MainThread INFO in loghandler: filepath /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/noteboo

/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/services/child_text_unify_to_parent.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[block_index]['text'] = str(text)


[2020-09-09 13:00:38,651] {loghandler.py:20} MainThread INFO in loghandler: Extracting xml of /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/010220_hi.pdf
[2020-09-09 13:00:38,764] {loghandler.py:20} MainThread INFO in loghandler: Extracting background images of /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/010220_hi.pdf
[2020-09-09 13:00:38,764] {loghandler.py:20} MainThread INFO in loghandler: Extraction of /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/010220_hi.pdf completed in 0.428663969039917
Total number of pages (2) in file (010220_hi.xml)
Total number of pages (2) in file (010220_hi.xml)
[2020-09-09 13:00:38,792] {loghandler.py:20} MainThread INFO in loghandler: process_input_pdf: created dataframes successfully
[2020-09-09 13:00:38,793] {loghandler.py:20} MainThread INFO in loghandler:

...

[2020-09-09 13:04:54,294] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_hdfs completed in 34.66783308982849/151, average per page 0.2295882986081357
[2020-09-09 13:05:02,100] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_vdfs completed in 7.8060619831085205/151, average per page 0.05169577472257298


/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


[2020-09-09 13:05:18,087] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_hdfs completed in 15.985734224319458/151, average per page 0.10586578956502952
[2020-09-09 13:05:19,147] {loghandler.py:20} MainThread INFO in loghandler: UnderLineDetection successfully completed
[2020-09-09 13:05:31,830] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 12.681725978851318/151, average per page 0.0839849402572935


/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/services/child_text_unify_to_parent.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[block_index]['text'] = str(text)


[2020-09-09 13:05:58,739] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 26.906378030776978/151, average per page 0.17818793397865548
[2020-09-09 13:05:58,739] {loghandler.py:20} MainThread INFO in loghandler: document structure analysis successfully completed
[2020-09-09 13:05:58,747] {loghandler.py:20} MainThread INFO in loghandler: document structure response started  ===>
[2020-09-09 13:06:21,270] {loghandler.py:20} MainThread INFO in loghandler: document structure response successfully completed
[2020-09-09 13:06:21,270] {loghandler.py:20} MainThread INFO in loghandler: DocumentStructure : successfully received blocks in json response
[2020-09-09 13:06:21,280] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing <module 'src.utilities.app_context' from '/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/app_context.py'>
[2020-09-09 13:06:21,281] {loghandle

In [7]:
app_context.application_context = dict({'task_id': 'BM-DUMMY'})
pdf_path   = os.path.join(input_dir, filename)

data       = main.DocumentStructure(app_context, filename, "en", base_dir=input_dir)


[2020-09-09 12:56:53,160] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing <module 'src.utilities.app_context' from '/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/app_context.py'>
[2020-09-09 12:56:53,162] {loghandler.py:20} MainThread INFO in loghandler: document preprocessing started ===>
[2020-09-09 12:56:53,172] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/20.03.2020_d544c04a-f26d-11ea-8f9d-acde48001122
[2020-09-09 12:56:53,173] {loghandler.py:20} MainThread INFO in loghandler: filepath /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/20.03.2020.pdf, working_dir /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/20.03.2020_d

/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


[2020-09-09 12:57:02,385] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_hdfs completed in 1.9928009510040283/13, average per page 0.1532923808464637
[2020-09-09 12:57:02,409] {loghandler.py:20} MainThread INFO in loghandler: UnderLineDetection successfully completed


/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[2020-09-09 12:57:03,830] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 1.4197828769683838/13, average per page 0.10921406745910645


/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/services/child_text_unify_to_parent.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[block_index]['text'] = str(text)


[2020-09-09 12:57:06,487] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 2.6560471057891846/13, average per page 0.20431131582993728
[2020-09-09 12:57:06,487] {loghandler.py:20} MainThread INFO in loghandler: document structure analysis successfully completed
[2020-09-09 12:57:06,489] {loghandler.py:20} MainThread INFO in loghandler: document structure response started  ===>
[2020-09-09 12:57:09,039] {loghandler.py:20} MainThread INFO in loghandler: document structure response successfully completed
[2020-09-09 12:57:09,039] {loghandler.py:20} MainThread INFO in loghandler: DocumentStructure : successfully received blocks in json response


In [67]:
def draw_bbox_image(draw,page_data):
    
    if page_data['images']:
        try:
            for image_block in page_data['images']:
                    top     = image_block["text_top"];         left   = image_block["text_left"];  
                    bottom  = top+image_block["text_height"];  right  = left+image_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='green')
            return draw
        except:
            pass

In [68]:
def draw_bbox_table(draw,page_data):
    
    if page_data['tables']:
        try:
            for table_block in page_data['tables']:
                    top     = table_block["text_top"];         left   = table_block["text_left"];  
                    bottom  = top+table_block["text_height"];  right  = left+table_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='blue')
            return draw
        except:
            pass

In [69]:
def draw_bbox_text(draw,page_data):
    
    if page_data['text_blocks']:
        try:
            for text_block in page_data['text_blocks']:
                top     = text_block["text_top"];         left   = text_block["text_left"];  
                bottom  = top+text_block["text_height"];  right  = left+text_block["text_width"]
                draw.rectangle(((left, top), (right,bottom)), outline='red')
            return draw
        except:
            pass

In [70]:
def draw_bbox_pdf(data,image_files,save_path):
    for page_no in range(len(data)):
        image_path = sorted(image_files)[page_no]
        page_data = data[page_no]
        page_width = page_data['page_width']; page_height = page_data['page_height']
        image  = Image.open(image_path)
        image  = image.resize((page_width, page_height))
        draw   = ImageDraw.Draw(image)
        draw  = draw_bbox_text(draw,page_data)
        draw  = draw_bbox_image(draw,page_data)
        draw  = draw_bbox_table(draw,page_data)
        save_filepath = os.path.join(save_path,image_path.split('images/')[1])
        image.save(save_filepath)
            

In [71]:
draw_bbox_pdf(response,pdf_image_paths,save_path)